# Demo for D-Wave on Braket: Structural Imbalance

In this tutorial we solve a structural imbalance problem using the D-Wave device on Amazon Braket.


__Social networks__ map relationships between people or organizations onto graphs, with
the people/organizations as nodes and relationships as edges. __Signed social networks__ map both friendly and
hostile relationships by assigning to edges either positive or negative values. Such networks are said to be __structurally balanced__ when they can be cleanly divided into two sets, with each set containing only friends, and all relations between these sets are hostile. The measure of __structural imbalance__ or __frustration__ for a signed social network, when it cannot be cleanly divided, is the minimum number of edges that violate the social rule, “the enemy of my friend is my enemy.”

Disclaimer: The code shown in this example has been taken from D-Wave tutorial available online [here](https://github.com/dwave-examples/structural-imbalance-notebook), with copyright to D-Wave Systems, Inc., licensed under the Apache License. The purpose of this example is to show how existing code using D-Wave's Ocean tool suite can easily be run on Amazon Braket, with minimal code changes, using the ```BraketDWaveSampler```.

## Imports and setup

In [1]:
!pip install jupyter_contrib_nbextensions==0.5.1
!pip install bokeh==0.12.15
!pip install autopep8

In [2]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

from dwave.system.composites import EmbeddingComposite

# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = f"amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

In [3]:
# session and device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
print('Device:', device)

Device: Device('name': DW_2000Q_6, 'arn': arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6)


## A Toy Example

In alignment with the D-Wave tutorial example [here](https://github.com/dwave-examples/structural-imbalance-notebook), we firstly demonstrate a toy example of a small social network with four people, then in the next section we will implement the solution to a real-world problem.

In the toy example, each of these four people is connected to the others with randomly-generated relationships, which we use a ```networkx``` graph to represent.

In [3]:
import networkx as nx
import random

G = nx.complete_graph(4)

# Randomly assign +1 or -1 relationship signs to all edges. Rename node 0 to Alice, 1 to Bob, etc
G.add_edges_from([(u, v, {'sign': 2*random.randint(0, 1) - 1}) for u, v in G.edges])
nx.relabel_nodes(G, {0: 'Alice', 1: 'Bob', 2: 'Eve', 3: 'Wally'}, copy=False)

print('Friendly relationships: \n\t' + '\n\t'.join(list(x + " & " + y for (x, y, sign) in G.edges(data='sign') if (sign == 1))))
print('Hostile relationships: \n\t' + '\n\t'.join(list(x + " & " + y for (x, y, sign) in G.edges(data='sign') if (sign == -1))))

Friendly relationships: 
	Alice & Bob
	Alice & Eve
	Bob & Eve
	Eve & Wally
Hostile relationships: 
	Alice & Wally
	Bob & Wally


### Setting Up a Solver

The code below sets up a D-wave solver as a QPU supported with Braket. The function ```EmbeddingComposite()``` implements the minor-mapping strategy, by mapping the problem graph to D-Wave system’s numerically indexed qubits.

In [4]:
sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')
sampler = EmbeddingComposite(sampler)

### Solving the Problem

Next, the `structural_imbalance` algorithm submits the model to a D-Wave system. It returns a partition of the social network into two colored sets and the frustrated edges.

In [5]:
import dwave_networkx as dnx

imbalance, bicoloring = dnx.structural_imbalance(G, sampler)

# Mark on the graph the returned frustrated edges and node set (color)  
for edge in G.edges:
    G.edges[edge]['frustrated'] = edge in imbalance
for node in G.nodes:
    G.nodes[node]['color'] = bicoloring[node]

print('Yellow set: \n\t' + '\n\t'.join(list(person for (person, color) in bicoloring.items() if (color == 0))))
print('Blue set: \n\t' + '\n\t'.join(list(person for (person, color) in bicoloring.items() if (color == 1))))
print('Frustrated relationships: \n\t' + '\n\t'.join(list(x + " & " + y for (x, y) in imbalance.keys())))

Yellow set: 
	Wally
Blue set: 
	Alice
	Bob
	Eve
Frustrated relationships: 
	Eve & Wally


Display the solution using a `draw` function that represents friendly interactions as green lines, hostile interactions as red lines, and frustration as dashed lines.

In [6]:
from helpers.draw import draw

draw(G, with_labels=True);

Loading BokehJS ...

# A Real-World Example

The real-world example with large data sets is taken from [Mapping Militant Organizations, Stanford University, last modified February 28, 2016, http://web.stanford.edu/group/mappingmilitants/cgi-bin/](http://web.stanford.edu/group/mappingmilitants/cgi-bin/). Firstly, we load data from the Stanford Militants Mapping Project into the ```networkx``` graph. 

In [7]:
from helpers.loader import global_signed_social_network

G = global_signed_social_network()

First, calculate imbalance on a selected region and time period, Syria 2014, by filtering on these attributes of the data now in graph G.

In [8]:
# Select the Syria subregion 
syria_groups = set()
for v, data in G.nodes(data=True):
    if 'map' not in data:
        continue
    if data['map'] in {'Syria', 'Aleppo'}:
        syria_groups.add(v)
S = G.subgraph(syria_groups)

# Filter by year
year = 2014
filtered_edges = ((u, v) for u, v, a in S.edges(data=True) if a['event_year'] <= year)
S = S.edge_subgraph(filtered_edges)

The resulting graph has nodes representing militant groups, with indexical labels, and edges with a "sign" attribute marking friendly or hostile relationships.

Show data associated with the first two nodes and edges:

In [9]:
print(list(S.nodes(data=True))[:2])
print(list(S.edges(data=True))[:2])

[(1, {'map': 'Aleppo'}), (645, {'map': 'Syria'})]
[(1, 361, {'sign': -1, 'event_id': '1443', 'event_type': 'riv', 'event_year': 2014, 'event_description': "Kata'ib Hezbollah worked with elements of the Kurdish peshmerga to defend the city of Amerli in Iraq from IS. KH has been fighting alongside Iraqi government forces in the war against IS since June 2014. While the groups have opposed each other in Iraq, there have been no confirmed reports of KH targeting IS in Syria. "}), (1, 661, {'sign': -1, 'event_id': '1865', 'event_type': 'riv', 'event_year': 2011, 'event_description': 'Jaysh al Sanadeed began targeting the Islamic State.'})]


Display the network using the `draw` function that represents friendly interactions as green lines, hostile interactions as red lines.

In [10]:
position = draw(S)

As in the previous section, we use the `structural_imbalance` algorithm to calculate the frustration of the network on a D-Wave system, this time on the Syrian 2014 network. 

In [11]:
imbalance, bicoloring = dnx.structural_imbalance(S, sampler)

for edge in S.edges:
    S.edges[edge]['frustrated'] = edge in imbalance
for node in S.nodes:
    S.nodes[node]['color'] = bicoloring[node]
    
print(list(S.nodes(data=True))[:2])
print(list(S.edges(data=True))[:2])

[(1, {'map': 'Aleppo', 'color': 1}), (645, {'map': 'Syria', 'color': 0})]
[(1, 361, {'sign': -1, 'event_id': '1443', 'event_type': 'riv', 'event_year': 2014, 'event_description': "Kata'ib Hezbollah worked with elements of the Kurdish peshmerga to defend the city of Amerli in Iraq from IS. KH has been fighting alongside Iraqi government forces in the war against IS since June 2014. While the groups have opposed each other in Iraq, there have been no confirmed reports of KH targeting IS in Syria. ", 'frustrated': False}), (1, 661, {'sign': -1, 'event_id': '1865', 'event_type': 'riv', 'event_year': 2011, 'event_description': 'Jaysh al Sanadeed began targeting the Islamic State.', 'frustrated': False})]


Redraw the network with the previous node positioning: nodes are now bicolored and dashed lines indicate frustrated edges.

In [12]:
draw(S, position);

Redraw the network with a new positioning that separates the two sets.

In [13]:
draw(S);

Finally, the following cell attempts to calculate structural imbalance on the entire data set of the Stanford Militants Mapping Project, which has over 200 variables. 

In [14]:
imbalance, bicoloring = dnx.structural_imbalance(G, sampler)

for edge in G.edges:
    G.edges[edge]['frustrated'] = edge in imbalance
for node in G.nodes:
    G.nodes[node]['color'] = bicoloring[node]
draw(G);

__DISCUSSION:__ This is a structural imbalance problem. For a group of nodes with either friendly or hostile relationships as edges, the goal is to divide the group into two sets, with each set containing only friends, and find the frustration relationships between nodes.